In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_categ_logement(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", {"id": "produit-tableau-LOG_T2"})

    if table is None:
        print(f"[⚠️] Aucun tableau trouvé à l’URL : {url}")
        return None

    rows = table.find_all("tr")
    data = []

    for row in rows:
        cells = row.find_all(["th", "td"])
        row_data = [cell.get_text(strip=True) for cell in cells]
        if row_data:
            data.append(row_data)

    df = pd.DataFrame(data[1:], columns=data[0])

    if "Catégorie ou type de logement" not in df.columns or "2021" not in df.columns or "2015" not in df.columns or "2010" not in df.columns:
        print(f"[⚠️] Données incomplètes dans : {url}")
        return None

    return df

In [2]:

# Lire par chunks
chunksize = 50
reader = pd.read_csv("communes-france-2025.csv", dtype={"code_insee": str}, low_memory=False, chunksize=chunksize)

data_concat = []

for chunk_index, df_chunk in enumerate(reader):
    # Supprimer première colonne si elle est inutile
    if df_chunk.columns[0].startswith("Unnamed"):
        df_chunk = df_chunk.drop(df_chunk.columns[0], axis=1)

    # Filtrer les villes > 20k habitants
    filtered_source = df_chunk[df_chunk["population"] >= 20000]

    for i in range(filtered_source.shape[0]):
        code_insee = filtered_source["code_insee"].iloc[i]
        ville = filtered_source["nom_sans_accent"].iloc[i]

        url = f"https://www.insee.fr/fr/statistiques/2011101?geo=COM-{code_insee}"
        print(f"[{chunk_index+1}] étape : {i+1}/{filtered_source.shape[0]} - {ville} ({code_insee})")

        recherche = get_categ_logement(url)
        if recherche is not None:
            recherche["code_insee"] = code_insee
            recherche["Ville"] = ville
            data_concat.append(recherche)

# Concat final
df_final = pd.concat(data_concat, ignore_index=True)
df_final.to_csv("data_log_T2.csv", index=False, encoding="utf-8-sig")

[1] étape : 1/1 - bourg-en-bresse (01053)
[5] étape : 1/1 - oyonnax (01283)
[16] étape : 1/1 - laon (02408)
[22] étape : 1/2 - saint-quentin (02691)
[22] étape : 2/2 - soissons (02722)
[28] étape : 1/1 - montlucon (03185)
[30] étape : 1/1 - vichy (03310)
[32] étape : 1/1 - manosque (04112)
[36] étape : 1/1 - gap (05061)
[38] étape : 1/4 - antibes (06004)
[38] étape : 2/4 - cagnes-sur-mer (06027)
[38] étape : 3/4 - cannes (06029)
[38] étape : 4/4 - le-cannet (06030)
[39] étape : 1/3 - grasse (06069)
[39] étape : 2/3 - mandelieu-la-napoule (06079)
[39] étape : 3/3 - menton (06083)
[40] étape : 1/2 - nice (06088)
[40] étape : 2/2 - saint-laurent-du-var (06123)
[41] étape : 1/1 - vallauris (06155)
[50] étape : 1/1 - charleville-mezieres (08105)
[71] étape : 1/1 - troyes (10387)
[73] étape : 1/1 - carcassonne (11069)
[77] étape : 1/1 - narbonne (11262)
[83] étape : 1/1 - millau (12145)
[84] étape : 1/1 - rodez (12202)
[86] étape : 1/4 - aix-en-provence (13001)
[86] étape : 2/4 - allauch (13